# भाग 8 बीआयएस - Protocols (प्रोटोकॉलची) ओळख

### संदर्भ

आता आपण योजनांवरुन आलो आहोत, आपण प्रोटोकॉल नावाची नवीन वस्तू(object) सादर करू. एक प्रोटोकॉल योजनांच्या अनुक्रमांचे समन्वय करतो, त्यांना दूरच्या कामगारांवर तैनात करतो आणि त्यांना एका पासमध्ये चालवितो.

हा एक उच्च स्तरीय ऑब्जेक्ट आहे ज्यात बर्‍याच कामगारांमध्ये वितरित केलेल्या कॉम्प्लेक्स संगणनाचे लॉजिक असते. प्रोटोकॉलचे मुख्य वैशिष्ट्य म्हणजे कामगारांमध्ये पाठविण्याची / शोधण्याची / परत आणण्याची क्षमता आणि शेवटी ओळखलेल्या कामगारांमध्ये तैनात करणे. म्हणून एखादा वापरकर्ता एक प्रोटोकॉल डिझाइन करू शकतो, क्लाउड वर्करवर अपलोड करू शकतो आणि इतर कोणत्याही कामगार ते शोधू शकतील, डाउनलोड करू शकतील आणि त्यास जोडलेल्या संगणकावर संगणकीय प्रोग्राम लागू करु शकतील.

चला कसे वापरायचे ते पाहूया!

लेखक:
- Théo Ryffel - Twitter [@theoryffel](https://twitter.com/theoryffel) - Github: [@LaRiffle](https://github.com/LaRiffle)

अनुवादक/संपादक:
- Krunal Kshirsagar - Twitter: [@krunal_wrote](https://twitter.com/krunal_wrote) - Github: [@Noob-can-Compile](https://github.com/Noob-can-Compile)



### 1. तयार करा आणि तैनात करा

प्रोटोकॉल जोड्यांची यादी `(worker, plan)` प्रदान करुन तयार केले जातात. `worker` एकतर खरा कामगार किंवा कामगार आयडी किंवा कल्पित कामगार प्रतिनिधित्त्व करण्यासाठी स्ट्रिंग असू शकतो. या शेवटच्या घटकाचा वापर निर्मितीवर वापरण्यासाठी वापरला जाऊ शकतो की उपयोजित वेळी त्याच योजनेद्वारे दोन योजना मालकीच्या (किंवा मालकीच्या नसल्या पाहिजेत). `plan` एकतर योजना किंवा PointerPlan असू शकते.

In [ ]:
import torch as th
import syft as sy
hook = sy.TorchHook(th)

# IMPORTANT: Local worker should not be a client worker
hook.local_worker.is_client_worker = False

चला 3 योजना परिभाषित करू आणि त्यांना एका प्रोटोकॉलमध्ये फीड करू. ते सर्व वाढीव ऑपरेशन करतात.

In [ ]:
@sy.func2plan(args_shape=[(1,)])
def inc1(x):
    return x + 1

@sy.func2plan(args_shape=[(1,)])
def inc2(x):
    return x + 1

@sy.func2plan(args_shape=[(1,)])
def inc3(x):
    return x + 1

protocol = sy.Protocol([("worker1", inc1), ("worker2", inc2), ("worker3", inc3)])

आता आपल्याला कर्मचार्‍यांना प्रोटोकॉल बाइंड करणे आवश्यक आहे, जे `.deploy (*workers)` कॉल करून केले जाते. चला काही कामगार तयार करूया.

In [ ]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
charlie = sy.VirtualWorker(hook, id="charlie")

In [ ]:
workers = alice, bob, charlie

protocol.deploy(*workers)

आपण पाहू शकता की योजना योग्य कामगारांना यापूर्वीच पाठविल्या गेल्या आहेत: त्या तैनात केल्या आहेत!
 
हे 2 टप्प्यात केले गेले आहे: प्रथम, आम्ही प्रदान केलेल्या कल्पित कामगारांचा नकाशा प्रदान केलेल्या (स्ट्रिंग्सद्वारे नावे दिलेला) कामगारांना आणि दुसरे म्हणजे आम्ही त्या प्रत्येकाला संबंधित योजना पाठवतो.

### 2. एक प्रोटोकॉल चालवा

प्रोटोकॉल चालवणे म्हणजे सर्व योजना क्रमाने कार्यान्वित करणे. असे करण्यासाठी आपण काही इनपुट डेटा प्रदान करा जो प्रथम योजनेच्या ठिकाणी पाठविला जाईल. ही पहिली योजना चालविली गेली आहे आणि त्याचे आउटपुट दुसर्‍या योजनेच्या ठिकाणी हलविले गेले आहे आणि याप्रमाणे. सर्व योजना संपल्यानंतर अंतिम निकाल परत केला जातो आणि तो शेवटच्या योजनेच्या ठिकाणी निर्देशकांचा बनलेला असतो.

In [ ]:
x = th.tensor([1.0])
ptr = protocol.run(x)
ptr

In [ ]:
ptr.get()

इनपुट 1.0 हे 3 योजनांच्या माध्यमातून केले गेले आहे आणि त्यानुसार 3 वेळा वाढ केली गेली आहे, म्हणूनच आता ते 4.0 च्या बरोबरीचे आहे!

प्रत्यक्षात, आपण काही पॉइंटर्सवर **दूरस्थपणे प्रोटोकॉल देखील** चालवू शकता:

In [ ]:
james = sy.VirtualWorker(hook, id="james")

In [ ]:
protocol.send(james)

In [ ]:
x = th.tensor([1.0]).send(james)
ptr = protocol.run(x)
ptr

तुम्ही पाहूचशक्ता निकाल जेम्सकडे निर्देशक आहे

In [ ]:
ptr = ptr.get()
ptr

In [ ]:
ptr = ptr.get()
ptr

### 3. प्रोटोकॉल शोधा

वास्तविक सेटिंग्जमध्ये आपणास रिमोट प्रोटोकॉल डाउनलोड करायचा असेल, तो आपल्या कामगारांवर तैनात करायचा असेल आणि आपल्याबरोबर डेटा चालवायचा असेलः

चला एक प्रोटोकॉल ** सुरू करू जो तैनात नाही ** आणि तो दूरस्थ कामगारांवर ठेवू

In [ ]:
protocol = sy.Protocol([("worker1", inc1), ("worker2", inc2), ("worker3", inc3)])
protocol.tag('my_protocol')
protocol.send(james)

In [ ]:
me = sy.hook.local_worker # get access to me as a local worker

आता आपण प्रोटोकॉल शोधण्यासाठी शोध सुरू करतो

In [ ]:
responses = me.request_search(['my_protocol'], location=james)
responses

आपल्याकडे प्रोटोकॉलच्या पॉईंटरवर प्रवेश आहे

In [ ]:
ptr_protocol = responses[0]

नेहमीच्या पॉईंटर प्रमाणे आपण ते परत मिळवू शकता:

In [ ]:
protocol_back = ptr_protocol.get()
protocol_back

आणि आपण भाग 1 आणि 2 प्रमाणे केले तसे करू शकतो.

In [ ]:
protocol_back.deploy(alice, bob, charlie)

x = th.tensor([1.0])
ptr = protocol_back.run(x)
ptr.get()

आणखी वास्तविक जगाची उदाहरणे प्रोटोकॉलसह येतील, परंतु आपण या नवीन ऑब्जेक्टद्वारे उघडलेल्या सर्व शक्यता आपण पहातच आहात!

### Pysyft ला Github वर Star करा!

आमच्या समुदायाला मदत करण्याचा सर्वात सोपा मार्ग म्हणजे फक्त गिटहब(GitHub) रेपो(Repo) तारांकित(Star) करणे! हे आम्ही तयार करीत असलेल्या छान साधनांविषयी जागरूकता वाढविण्यास मदत करते.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### GitHub वरील आमचे प्रशिक्षण निवडा.

आम्ही फेडरेटेड आणि गोपनीयता-संरक्षित लर्निंगबद्दल अधिक चांगल्या प्रकारे समजवण्यासाठी खरोखर चांगले ट्यूटोरियल बनवले आहेत.

- [PySyft ट्यूटोरियल्सला चेक करा](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials)

### आमच्या Slack मध्ये सामील व्हा!


नवीनतम प्रगतीवर अद्ययावत राहण्याचा उत्तम मार्ग म्हणजे आमच्या समुदायामध्ये सामील होणे! आपण [http://slack.openmined.org](http://slack.openmined.org) येथे फॉर्म भरुन तसे करू शकता.

### एका कोड प्रोजेक्टमध्ये सामील व्हा!

आमच्या समुदायामध्ये योगदानाचा उत्तम मार्ग म्हणजे कोड योगदानकर्ता बनणे! कोणत्याही वेळी आपण (PySyft GitHub Issues Page) वर  जाऊ शकता आणि "Project" साठी फिल्टर करू शकता. हे आपण कोणत्या प्रकल्पांमध्ये सामील होऊ शकता याबद्दल विहंगावलोकन देणारी सर्व उच्च स्तरीय तिकिटे दर्शवेल! आपण एखाद्या प्रकल्पात सामील होऊ इच्छित नसल्यास, परंतु आपण थोडं कोडिंग करू इच्छित असाल तर आपण `Good First Issue` म्हणून चिन्हांकित गिटहब(GitHub) अंक शोधून आणखी "one off" मिनी-प्रकल्प(mini project) शोधू शकता.

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### दान करा

आपल्याकडे आमच्या कोडेबेसमध्ये योगदान देण्यास वेळ नसल्यास, परंतु तरीही आपल्याला समर्थन द्यावयाचे असल्यास आपण आमच्या मुक्त संग्रहात बॅकर देखील होऊ शकता. सर्व देणगी आमच्या वेब होस्टिंग आणि हॅकॅथॉन आणि मेटअप्स सारख्या इतर सामुदायिक खर्चाकडे जातात!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)